## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
import sys
sys.path.append("..")
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Lockport,US,41.59,-88.06,88.00,55,1,16.11,Clear
1,1,Mataura,NZ,-46.19,168.86,45.00,72,71,11.01,Rain
2,2,Vestmannaeyjar,IS,63.44,-20.27,50.34,78,0,13.00,Clear
3,3,Yar-Sale,RU,66.83,70.83,52.27,90,27,5.57,Clouds
4,4,Tuktoyaktuk,CA,69.45,-133.04,48.20,76,20,10.29,Clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Lockport,US,41.59,-88.06,88.00,55,1,16.11,Clear
5,5,Candido Mendes,BR,-1.45,-45.72,76.19,88,11,4.65,Clouds
6,6,Porto Walter,BR,-8.27,-72.74,78.06,50,38,2.59,Clouds
8,8,Portland,US,45.52,-122.68,82.99,34,1,6.93,Clear
9,9,Ixtapa,MX,20.70,-105.20,86.00,74,90,8.05,Clouds
15,15,Hilo,US,19.73,-155.09,84.20,62,1,9.17,Clear
17,17,Los Llanos De Aridane,ES,28.66,-17.92,77.00,88,5,9.17,Clear
18,18,Sungairaya,ID,0.70,108.90,78.42,83,55,8.46,Clouds
22,22,Victoria,HK,22.29,114.16,80.01,90,90,1.99,Rain
25,25,Mazatlan,MX,23.22,-106.42,89.01,66,75,9.17,Clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
717,False,False,False,False,False,False,False,False,False,False
718,False,False,False,False,False,False,False,False,False,False
720,False,False,False,False,False,False,False,False,False,False
728,False,False,False,False,False,False,False,False,False,False


In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Lockport,US,88.00,Clear,41.59,-88.06,
5,Candido Mendes,BR,76.19,Clouds,-1.45,-45.72,
6,Porto Walter,BR,78.06,Clouds,-8.27,-72.74,
8,Portland,US,82.99,Clear,45.52,-122.68,
9,Ixtapa,MX,86.00,Clouds,20.70,-105.20,
15,Hilo,US,84.20,Clear,19.73,-155.09,
17,Los Llanos De Aridane,ES,77.00,Clear,28.66,-17.92,
18,Sungairaya,ID,78.42,Clouds,0.70,108.90,
22,Victoria,HK,80.01,Rain,22.29,114.16,
25,Mazatlan,MX,89.01,Clouds,23.22,-106.42,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
hotel_df.head(10)

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Lockport,US,88.00,Clear,41.59,-88.06,Big Run Wolf Ranch
5,Candido Mendes,BR,76.19,Clouds,-1.45,-45.72,Quebrada Do Mlk PJL
6,Porto Walter,BR,78.06,Clouds,-8.27,-72.74,Hotel Silva
8,Portland,US,82.99,Clear,45.52,-122.68,Portland Marriott City Center
9,Ixtapa,MX,86.00,Clouds,20.70,-105.20,Container Inn Puerto Vallarta
15,Hilo,US,84.20,Clear,19.73,-155.09,Hilo Hawaiian Hotel
17,Los Llanos De Aridane,ES,77.00,Clear,28.66,-17.92,Valle Aridane
18,Sungairaya,ID,78.42,Clouds,0.70,108.90,808 artel
22,Victoria,HK,80.01,Rain,22.29,114.16,Mini Hotel Central
25,Mazatlan,MX,89.01,Clouds,23.22,-106.42,Gaviana Resort


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Lockport,US,88.00,Clear,41.59,-88.06,Big Run Wolf Ranch
5,Candido Mendes,BR,76.19,Clouds,-1.45,-45.72,Quebrada Do Mlk PJL
6,Porto Walter,BR,78.06,Clouds,-8.27,-72.74,Hotel Silva
8,Portland,US,82.99,Clear,45.52,-122.68,Portland Marriott City Center
9,Ixtapa,MX,86.00,Clouds,20.70,-105.20,Container Inn Puerto Vallarta
...,...,...,...,...,...,...,...
717,Hirara,JP,86.00,Rain,24.80,125.28,Hotel Atoll Emerald Miyakojima
718,Porto Novo,BJ,75.20,Clouds,6.50,2.60,Tour Eiffel Hotel Benin
720,Rabo De Peixe,PT,75.20,Rain,37.80,-25.58,Quinta das Giestas
728,Bathsheba,BB,82.40,Clouds,13.22,-59.52,Atlantis Hotel


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))